In [15]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import imodelsx.metrics
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import numpy as np

import imodelsx.treeprompt.stump

from tprompt.compiler.evaluator import PromptHooker, modify_activations
from tprompt.compiler import compiling

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load prompt vector

In [16]:
# hyperparams
checkpoint = 'gpt2'
prompts = [
    " This review of a movie is",
    " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    # " The plot of the movie was really",
    " The acting in the movie was",

]
prompt_at_start_or_end = 'start'
# note: also requires specifying the layer for the hook below

In [17]:
avg_soft_prompt = compiling.get_avg_soft_prompt(checkpoint, prompts)

In [20]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained(checkpoint)

# tokenize the prompts

In [27]:
lens = [len(tok.encode(prompt)) for prompt in prompts]

# get the index of the longest prompt
max_prompt_len = np.argmax(lens)

In [28]:
lens

[6, 7, 6, 6]

# Evaluate accuracies

In [9]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=1000, replace=False))
# dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
# dset_val = dset_val.select(np.random.choice(
#     len(dset_val), size=100, replace=False))

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
verbalizer = {0: " Negative.", 1: " Positive."}

m = PromptHooker(
    checkpoint=checkpoint,
    # this should probably be the prompt with the max num tokens?
    prompts=[prompts[0]],
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=avg_soft_prompt,
    prompt_at_start_or_end='end',
    prompt_template="{example}{prompt}",
)
m.fit(dset_train["text"], dset_train["label"])
acc_avg = m.prompt_accs_[0]

print('------------------------')
m = PromptHooker(
    checkpoint=checkpoint,
    prompts=prompts,
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=None,
    prompt_at_start_or_end='end',
)
m.fit(dset_train["text"], dset_train["label"])
accs0 = deepcopy(m.prompt_accs_)
print('**Averaged** ->', acc_avg)
for i, prompt in enumerate(prompts):
    print(i, prompt, '->', accs0[i])

Prompt 0:  This review of a movie is


------------------------
Prompt 0:  This review of a movie is


Prompt 1:  Positive or Negative? The movie was


Prompt 2:  The sentiment of the movie was


Prompt 3:  The acting in the movie was


**Averaged** -> 0.549
0  This review of a movie is -> 0.483
1  Positive or Negative? The movie was -> 0.489
2  The sentiment of the movie was -> 0.54
3  The acting in the movie was -> 0.519
